In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 开始使用BigQuery ML培训

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

概述

本教程演示了如何在生产环境中使用Vertex AI。本教程涵盖了如何开始使用BigQuery ML进行训练。

了解更多关于[BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml)。

### 目标

在本教程中，您将学习如何使用BigQuery ML在Vertex AI上进行训练。

此教程使用以下Google Cloud ML服务：

- BigQuery ML 训练
- Vertex AI 模型资源
- Vertex AI Vizier

执行的步骤包括：

- 在您的项目中创建一个本地的BigQuery表
- 训练一个BigQuery ML模型
- 评估BigQuery ML模型
- 将BigQuery ML模型导出为云模型
- 将导出的模型上传为Vertex AI模型资源
- 使用Vertex AI Vizier对BigQuery ML模型进行超参数调整
- 自动将BigQuery ML模型注册到Vertex AI模型注册表

### 数据集

本教程使用的数据集来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)中的企鹅数据集。该数据集的这个版本用于根据可用特征（如喙长度、鳍深度等）来预测企鹅的物种。

### 成本
本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- 云存储
- BigQuery

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)、[云存储定价](https://cloud.google.com/storage/pricing) 和 [BigQuery 定价](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet pyarrow \
                                 google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 google-cloud-bigquery-storage \
                                 db-dtypes 

只有协作：请取消对以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关说明操作。

**1. Vertex AI Workbench**
* 无需做任何操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作, 取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在以下网址 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

#### 服务账号

您可以使用服务账号来创建Vertex AI管道作业。如果您不想使用项目的计算引擎服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### 初始化Python的Vertex AI和BigQuery SDK

为你的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

创建 BigQuery 客户端。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### 设置硬件加速器

您可以为预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像和分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个 GPU 容器映像，每个 VM 配置有 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aiplatform.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个运行在 CPU 上的容器映像。

在此处了解更多有关您所在地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
import os

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

### 设置预构建容器

为预测设置预构建的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建镜像：

要查看最新列表，请访问[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
   - `机器类型`
       - `n1-standard`：每个vCPU 3.75GB的内存。
       - `n1-highmem`：每个vCPU 6.5GB的内存
       - `n1-highcpu`：每个vCPU 0.9GB的内存
   - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个处理器核心

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

BigQuery ML简介

BigQuery ML（简称BQML）提供了在BigQuery中使用SQL语法训练ML表格模型（如分类和回归）的功能。

了解更多关于[BigQuery ML文档](https://cloud.google.com/bigquery-ml/docs)。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

创建BQ数据集资源

首先，在您的项目中创建一个空的数据集资源。

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}
"""

job = bqclient.query(DATASET_QUERY)

### 训练 BigQuery ML 模型

接下来，您将从公共数据集 penguins 创建并训练一个 BigQuery ML 表格分类模型，并使用 `CREATE MODEL` 语句将模型存储在您的项目中。模型配置在 `OPTIONS` 语句中按如下方式指定：

- `model_type`：要训练的表格模型的类型和架构，例如 DNN 分类。
- `labels`：标签列。

了解有关[CREATE MODEL 语句](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)的更多信息。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species']
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### 评估经过训练的BigQuery ML模型

接下来，检索受过训练的BigQuery ML模型的模型评估。

了解更多关于[ML.EVALUATE函数](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)。

In [ ]:
EVAL_QUERY = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
ORDER BY  roc_auc desc
LIMIT 1"""

try:
    job = bqclient.query(EVAL_QUERY)
    results = job.result().to_dataframe()
    print(results)
except Exception as e:
    print(e)

### 从BigQuery ML导出模型

您在BigQuery ML中训练的模型是一个TensorFlow模型。接下来，您将以TF.SavedModel格式导出TensorFlow模型的工件。

In [ ]:
param = f"{PROJECT_ID}:{BQ_DATASET_NAME}.{MODEL_NAME} {BUCKET_URI}/{MODEL_NAME}"
! bq extract -m $param

MODEL_DIR = f"{BUCKET_URI}/{BQ_DATASET_NAME}"
! gsutil ls $MODEL_DIR

## 将BigQuery ML模型上传到Vertex AI Model资源

最后，现在您已经导出了BigQuery ML模型，您可以将模型工件上传到Vertex AI Model资源，方式与上传自定义训练模型相同。

以下是将BigQuery ML模型类型映射到其相应导出模型格式的部分列表：

'LINEAR_REG'<br/>
'LOGISTIC_REG' --> TensorFlow SavedFormat

'AUTOML_CLASSIFIER'<br/>
'AUTOML_REGRESSOR' --> TensorFlow SavedFormat

'BOOSTED_TREE_CLASSIFIER'<br/>
'BOOSTED_TREE_REGRESSOR' --> XGBoost格式

'DNN_CLASSIFIER'<br/>
'DNN_REGRESSOR'<br/>
'DNN_LINEAR_COMBINED_CLASSIFIER'<br/>
'DNN_LINEAR_COMBINED_REGRESSOR' --> TensorFlow Estimator

In [ ]:
try:
    model = aiplatform.Model.upload(
        display_name="penguins",
        artifact_uri=MODEL_DIR,
        serving_container_image_uri=DEPLOY_IMAGE,
        sync=True,
    )
except Exception as e:
    print(e)

## 部署模型

接下来，为了进行在线预测，部署您的模型。要部署模型，您需要调用`deploy`方法，并设置以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：在端点的流量中流向该模型的百分比，指定为一个或多个键值对的字典。
如果只有一个模型，请设置为{ "0": 100 }，其中"0"指的是上传的模型，100表示100%的流量。
如果存在需要分配流量的现有模型，则使用`model_id`指定为{ "0": 百分比, model_id: 百分比, ... }，其中`model_id`是部署终端的现有模型的模型ID。各百分比必须总和为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作复制品的加速器数量。
- `starting_replica_count`：最初要提供的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，只有一个实例被预置。

In [ ]:
DEPLOYED_NAME = "penguins"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

try:
    if DEPLOY_GPU:
        endpoint = model.deploy(
            deployed_model_display_name=DEPLOYED_NAME,
            traffic_split=TRAFFIC_SPLIT,
            machine_type=DEPLOY_COMPUTE,
            accelerator_type=DEPLOY_GPU.name,
            accelerator_count=DEPLOY_NGPU,
            min_replica_count=MIN_NODES,
            max_replica_count=MAX_NODES,
        )
    else:
        endpoint = model.deploy(
            deployed_model_display_name=DEPLOYED_NAME,
            traffic_split=TRAFFIC_SPLIT,
            machine_type=DEPLOY_COMPUTE,
            min_replica_count=MIN_NODES,
            max_replica_count=MAX_NODES,
        )
except Exception as e:
    print(e)

取消部署模型

当您完成预测时，您可以取消部署模型从`Endpoint`资源中。 这将取消所有计算资源并结束对部署模型的计费。

In [ ]:
try:
    endpoint.undeploy_all()
except Exception as e:
    print(e)

删除 `Vertex AI Model` 资源

方法 `delete()` 用于删除该模型。

In [ ]:
try:
    model.delete()
except Exception as e:
    print(e)

### 删除 `BigQuery ML` 模型

接下来，删除模型的 `BigQuery ML` 实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

try:
    job = bqclient.query(MODEL_QUERY)
except Exception as e:
    print(e)

### 超参数调整和训练 BigQuery ML 模型

接下来，您可以使用 Vertex AI Vizier 服务进行超参数调整来训练 BigQuery ML 表格分类模型。超参数设置在 `OPTIONS` 语句中指定如下：

- `HPARAM_TUNING_ALGORITHM`：选择下一个试验参数的算法。
- `num_trials`：试验次数。
- `max_parallel_trials`：并行进行试验的数量。

了解更多关于[CREATE MODEL 语句的超参数调整](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning)。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    num_trials=10,
    max_parallel_trials=2,
    HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

评估BigQuery ML训练好的模型

接下来，检索已训练的BigQuery ML模型的模型评估结果。

了解更多关于[ML.EVALUATE函数](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)。

In [ ]:
try:
    EVAL_QUERY = f"""
    SELECT *
    FROM
      ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
    ORDER BY  roc_auc desc
    LIMIT 1"""

    job = bqclient.query(EVAL_QUERY)
    results = job.result().to_dataframe()
    print(results)
except Exception as e:
    print(e)

删除 BigQuery ML 模型

接下来，删除模型的 BigQuery ML 实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

### 使用可解释性训练一个BigQuery ML模型

接下来，您使用相同的BigQuery ML模型进行训练，但这次您通过添加选项`ENABLE_GLOBAL_EXPLAIN`来启用顶点AI可解释性对模型预测进行解释。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    ENABLE_GLOBAL_EXPLAIN = True
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### 删除BigQuery ML模型

接下来删除该模型的BigQuery ML实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

## 模型注册表

或者，您可以通过导出和导入模型工件将您的BigQuery ML模型隐式上传为Vertex AI模型资源。在这种方法中，您在训练模型时添加额外选项，告诉BigQuery ML自动上传和注册训练模型作为模型资源。

### 设置权限以自动注册模型

您需要为BigQuery ML设置一些额外的IAM权限，以便在训练后自动上传和注册模型。根据您的服务帐户，下面权限的设置可能会失败。在这种情况下，我们建议在Cloud Shell中执行权限。

了解更多关于[设置模型注册表权限](https://cloud.google.com/bigquery-ml/docs/managing-models-vertex)。

In [ ]:
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT --role=roles/aiplatform.admin --condition=None

### 训练并注册模型

接下来，您可以通过添加以下参数选项，来训练模型并自动注册到 Vertex AI 模型注册表中：

- `model_registry`: 设置为 "vertex_ai"，表示自动注册到 Vertex AI 模型注册表。
- `vertex_ai_model_id`: 注册模型的人类可读的显示名称。
- `vertex_ai_model_version_aliases`: 模型的备用名称。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    model_registry="vertex_ai",
    vertex_ai_model_id="bqml_model", 
    vertex_ai_model_version_aliases=["1"]
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### 在Vertex Model Registry中查找模型

最后，您可以使用Vertex AI Model `list()`方法并附带筛选查询来查找自动注册的模型。

In [ ]:
models = aiplatform.Model.list(filter="display_name=bqml_model")
model = models[0]

print(model.gca_resource)

In [ ]:
models = aiplatform.Model.list()
for model in models:
    if model.gca_resource.display_name.startswith("bqml"):
        print(model.gca_resource.display_name)

删除BigQuery ML模型

接下来，删除该模型的BigQuery ML实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

将 `delete_storage` 设置为 `True` 来删除此笔记本中使用的云存储存储桶。

In [ ]:
# Delete the endpoint using the Vertex endpoint object
try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

# Delete the model using the Vertex model object
try:
    model.delete()
except Exception as e:
    print(e)

# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME

delete_storage = False
if delete_storage or os.getenv("IS_TESTING"):
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI